In [366]:
import math
import collections as col
import copy
import numpy as np

In [256]:
n = 4
min_shadow = [[1,1,1,1],[1,1,1,0],[0,1,1,1],[1,1,0,1],[1,1,0,0],[0,0,1,1],[1,0,0,0]]
f = '1100110011000111'

In [2]:
def findZhegalkin(inputVec):
    n = int(math.log2(len(inputVec)))
    zhigalkin = inputVec.copy()

    # calculate zhigalkin
    for i in range(n):
        gap = 2 ** i
        j = gap
        while j < len(zhigalkin):
            for l in range(gap):
                zhigalkin[j] = (zhigalkin[j] + zhigalkin[j - gap]) % 2
                j += 1

            j += gap

    return zhigalkin

In [153]:
result_poli = parseZhigalkin(findZhegalkin([int(x) for x in f]))
print(result_poli)

[1, [0, 0, 1, 0], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 1]]


In [455]:
inp_vec = [int(x) for x in f]
print(inp_vec)

[1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1]


In [9]:
def parseZhigalkin(zhigalkin):
    pol = []
    n = int(math.log2(len(zhigalkin)))

    for i in range(len(zhigalkin)):
        if zhigalkin[i] == 1:
            binary = decToBin(i, n)

            if sum(binary) == 0:
                pol += [1]
            else:
                pol += [binary]

    return pol


In [11]:
def decToBin(dec, n):
    result = []

    while True:
        result += [dec % 2]
        dec //= 2

        if dec == 0:
            break

    if len(result) != n:
        for i in range(n - len(result)):
            result += [0]

    return result[::-1]

In [496]:
def logic_minimize(s):
    poli = copy.deepcopy(s)
    d = col.defaultdict(list)
    #generate dict {rank:list_of_cons}, f.e. {0: [1], 1: [[0, 0, 1, 0]], 2: [[1, 1, 0, 0]], 3: [[1, 1, 0, 1]], 4: [[1, 1, 1, 1]]}
    for x in poli:
        if (x == 1): d[con_rank(x)].append(x)
        else: d[con_rank(x)].append(x)
    z = 0
    while (z != len(poli)):
        #print('start', z, poli)
        x = poli[z]
    #for x in poli:
        if (x == 1):
            if (1 in d.keys()):
                #print(True) 
                #optimization has found 
                #1. change dict, delete 0-key
                del d[0]
                #2. delete con from poli
                poli.remove(x)
                #3. change poli
                ind = poli.index(d[1][0])
                not_zero_ind = find_not_zero_index(d[1][0])
                if (poli[ind][not_zero_ind] == 1): poli[ind][not_zero_ind] = 2
                elif (poli[ind][not_zero_ind] == 2): poli[ind][not_zero_ind] = 1
                else: print('error1')
                #here is needed to restart cycle with updated x
                z -= 1
        else:
            for i in range(len(x)):
                if (x[i] == 0): continue
                new = x[:i] + [0] + x[i+1:]
                if (new in d[con_rank(x) - 1]):
                    #print(True)
                    #optimization has found
                    #1. change poli
                    ind = poli.index(x)
                    if (poli[ind][i] == 1): poli[ind][i] = 2
                    elif (poli[ind][i] == 2): poli[ind][i] = 1
                    else: print('error1')
                    #2. change dict
                    d[con_rank(x) - 1].remove(new) #delete subcon
                    d[con_rank(x)][d[con_rank(x)].index(x)] = poli[ind]
                    #3. delete con from poli
                    poli.remove(new)
                    #here is needed to restart cycle with updated x
                    z -= 1
                    break
        #print('end', z, poli)
        z += 1
    return poli

In [493]:
def con_rank(con): #con - conjunction: [1, 0, 2, 1, 0, 1] = x1*~x3*x4*x6 - rank = 4 (num of variables)
    rank = 0
    if (con == 1): return 0 
    for x in con:
        if (x != 0): rank += 1
    return rank

In [154]:
def find_not_zero_index(con):
    for i in range(len(con)):
        if (con[i] != 0): return i

In [304]:
def check_shadow_full(shadow):
    n = len(shadow[0])
    targ = 2**n - 1
    s = set()
    for x in shadow:
        for i in range(len(x)):
            if (x[i] == 0): continue
            else:
                x[i] = 0
                s.add(tuple(x))
                x[i] = 1
    #return len(s)
    if (len(s) == targ): return True
    else: return False
        

In [258]:
temp = copy.deepcopy(min_shadow)
check_shadow_full(temp)

15

In [338]:
def gen_min_shadow(n):
    shadow = []
    rank = n
    s = set() 
    allvec = []
   
   #generate all rank collections
   
    for i in range(2**n - 1, -1, -1):
        allvec.append(decToBin(i, n))
    #sort by rank
    all_sorted = sorted(allvec, key = con_rank, reverse = True)
    #cycle
    sum = 0
    for k in range(n):
        one_rank_vec = all_sorted[sum:sum+math.comb(n,k)]
        for its in range(len(one_rank_vec)):
            #now check whole set and create dict {len(locla_s):{shadow_vec}}
            d_vec = col.defaultdict(list)
            for vec in one_rank_vec:
                sh = vec_shadow(vec)
                len_delta = len(s | sh) - len(s)
                d_vec[len_delta].append(vec)
            #find the best
            max_key = max(d_vec.keys())
            if (max_key == 0): break
            shadow.append(d_vec[max_key][0])
            for x in vec_shadow(d_vec[max_key][0]):
                s.add(tuple(x))
        sum += math.comb(n,k)
        if (check_shadow_full(shadow)): break
    #print(shadow)
    #print(s)
    #print(check_shadow_full(shadow))
    return shadow  

In [339]:
gen_min_shadow(4)

[[1, 1, 1, 1],
 [1, 1, 1, 0],
 [1, 1, 0, 1],
 [1, 0, 1, 1],
 [1, 1, 0, 0],
 [0, 0, 1, 1],
 [1, 0, 0, 0]]

In [ ]:
s = {(1, 0, 1), (1, 1, 0), (0, 1, 0), (1, 0, 0), (0, 1, 1)}
d_vec = {}
for vec in one_rank_vec:
    sh = vec_shadow(vec)
    len_delta = len(s | sh) - len(s)
    d_vec[len_delta].append(vec)
print(d_vec)

In [303]:
m = [[1,1,1,1,1], [1,1,1,1,0], [1,1,1,0,1], [1,1,0,1,1], [0,1,1,1,1], [1,1,1,0,0], [1,1,0,1,0],[1,0,1,0,1], [0,1,0,1,1], [0,0,1,1,1], [1,1,0,0,0], [1,0,1,0,0],[0,0,0,1,1],[1,0,0,0,0]]
check_shadow_full([[1,1,1,1]])

4

In [261]:
a = sorted(m, key = con_rank, reverse = True)
a

[[1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0],
 [1, 1, 1, 0, 1],
 [1, 1, 0, 1, 1],
 [0, 1, 1, 1, 1],
 [1, 1, 1, 0, 0],
 [1, 1, 0, 1, 0],
 [1, 0, 1, 0, 1],
 [0, 1, 0, 1, 1],
 [0, 0, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 0, 1, 0, 0],
 [0, 0, 0, 1, 1],
 [1, 0, 0, 0, 0]]

In [312]:
def vec_shadow(vector):
    result = set()
    for i in range(len(vector)):
        if (vector[i] == 1):
            result.add(tuple(vector[:i] + [0] + vector[i+1:]))
    return result

In [314]:
s = set()
vecsres = vec_shadow([1,1,1,1])
print(vecsres)

{(1, 1, 0, 1), (1, 1, 1, 0), (0, 1, 1, 1), (1, 0, 1, 1)}


### Algorithm

In [479]:
n = 3
T_min_set = gen_min_shadow(n)

In [480]:
f = '1100110011000111'
input_vec = [int(x) for x in f]
inp_vec = input_vec = value_by_poly([[1,1,1],[1,1,0],[1,0,1],[1,0,0],[0,0,1],1])

In [481]:
zheg_input_vec = parseZhigalkin(findZhegalkin(input_vec))

In [482]:
F_Tshadow_poli = T_min_set #format: x1*x2*x3 + x1*x2 + x1*x4 + x2 #Phi

In [483]:
print(F_Tshadow_poli, '\n', zheg_input_vec)

[[1, 1, 1], [1, 1, 0], [1, 0, 1], [1, 0, 0]] 
 [1, [0, 0, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]


In [484]:
#step 3
#Psi
control_set = set([tuple(x) for x in F_Tshadow_poli if x != 1]) ^ set([tuple(x) for x in zheg_input_vec if x != 1])
if ((1 in F_Tshadow_poli) ^ (1 in zheg_input_vec)): control_set.add(1)
control_set = list(control_set)

In [485]:
#step 4-5
result = [] #delta
if (tuple([1]*n) in control_set):
    result.append(tuple([1]*n))
    set.remove(tuple([1]*n))

In [486]:
#step 6 always done
#step 7-8
for shadow in T_min_set:
    K = copy.deepcopy(shadow)
    K1 = copy.deepcopy(shadow)
    low_rank_list = [x for x in control_set if (con_rank(x) == con_rank(shadow)-1)]
    if (low_rank_list == [1]):
        K1[shadow.index(1)] = 2
    else:
        for x in low_rank_list:
            sub = (np.array(shadow) - np.array(x)).tolist()
            if (con_rank(sub) == 1 and 1 in sub):
                K1[sub.index(1)] = 2
    #step 8
    result.append(K1)
    control_set.append(tuple(K))
    control_set.append(tuple(K1))
    control_set = parseZhigalkin(findZhegalkin(value_by_poly(control_set)))
print(result)
print(value_by_poly(result))
if (value_by_poly(result) == inp_vec): print("True")
else: print("False")

[[1, 1, 1], [1, 1, 0], [2, 0, 1], [2, 0, 0]]
[1, 0, 1, 0, 0, 0, 1, 0]
True


In [474]:
def value_by_poly(poly): #list of lists/tuples
    if (1 in poly):
        poly.remove(1)
        if (len(poly) == 0): return 1
        result = np.array([1]*2**len(poly[0]), dtype = int)
    else:
        result = np.zeros(2**len(poly[0]), dtype = int)
    for x in poly:
        result = np.array(truthVector(x)) ^ result
    return result.tolist()

In [475]:
#must be optimized
def v(con, arg):
        v = 1
        check = 0

        for i in range(len(con)):
            check += con[i]

            if con[i] == 0:
                continue

            if con[i] == 1:
                v *= arg[i]
                continue

            v *= (arg[i] + 1) % 2

        if check == 0:
            return 0

        return v
def truthVector(con):
        n = len(con)
        vector = []

        for i in range(2 ** n):
            vector += [v(con, decToBin(i, n))]

        return vector

In [489]:
value_by_poly(result)

[1, 0, 1, 0, 0, 0, 1, 0]

In [498]:
logic_minimize(result)

[[1, 1, 2], [2, 0, 2]]